In [17]:
#Run with python3

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import datetime
import os
import string
import re

In [158]:
def fetch(url):
    req=Request(url, headers={'User-Agent': 'Misc Robot'})
    html_week = urlopen(req).read()
    soup = BeautifulSoup(html_week, 'html.parser')
    return soup

In [222]:
class Catalog:
    def __init__(self):
        self.base_url='http://usa.newonnetflix.info/catalog/a2z/comedies'
        self.pages=['']+list(string.ascii_lowercase)
        self.object_urls=[]
        
    def get_num(text):
        return re.findall(r"[0-9]*[0-9]", text)[0]
    
    def fetch_all(self):
        for page in self.pages:
            url=self.base_url+'/'+page
            print(url)
            soup = fetch(url)
            self.parse(soup)
        print(len(self.object_urls))
            
    def parse(self, soup):
        self.num_objects=parse_category_number(soup)
        articles=soup.find_all('section')[3].find_all('article')
        for article in articles:
            self.object_urls.append(article.find_all('a')[1]['href'])
            
    def parse_film_number(soup):
        total=soup.find('section').find_all('em')[0].text
        return get_num(total)

    def parse_category_number(soup):
        total=soup.find('section').find_all('em')[1].text
        return get_num(total)
        
        
    def __repr__(self):
        return self.base_url

In [223]:
netflix=Catalog()
netflix.fetch_all()

http://usa.newonnetflix.info/catalog/a2z/comedies/
http://usa.newonnetflix.info/catalog/a2z/comedies/a
http://usa.newonnetflix.info/catalog/a2z/comedies/b
http://usa.newonnetflix.info/catalog/a2z/comedies/c
http://usa.newonnetflix.info/catalog/a2z/comedies/d
http://usa.newonnetflix.info/catalog/a2z/comedies/e
http://usa.newonnetflix.info/catalog/a2z/comedies/f
http://usa.newonnetflix.info/catalog/a2z/comedies/g
http://usa.newonnetflix.info/catalog/a2z/comedies/h
http://usa.newonnetflix.info/catalog/a2z/comedies/i
http://usa.newonnetflix.info/catalog/a2z/comedies/j
http://usa.newonnetflix.info/catalog/a2z/comedies/k
http://usa.newonnetflix.info/catalog/a2z/comedies/l
http://usa.newonnetflix.info/catalog/a2z/comedies/m
http://usa.newonnetflix.info/catalog/a2z/comedies/n
http://usa.newonnetflix.info/catalog/a2z/comedies/o
http://usa.newonnetflix.info/catalog/a2z/comedies/p
http://usa.newonnetflix.info/catalog/a2z/comedies/q
http://usa.newonnetflix.info/catalog/a2z/comedies/r
http://usa.ne

In [241]:
class Article():
    def __init__(self, url):
        self.url='http://usa.newonnetflix.info'+url
    
    def build(self):
        soup=fetch(self.url)
        self.parse(soup)
        return self.attrs
    
    def parse(self, soup):
        article=soup.find_all('article')[0]
        attrs={}
        attrs['img_src']=article.find_all('img')[0]['src']
        attrs['genre']=article.select('div[class=genre]')[0].text
        
        p=article.find_all('p')

        
        for i in range(0,len(p)):
            if p[i].find('strong'):


                attr=p[i].find('strong').text[:-1]
                if attr not in ['IMDB', 'IMDB Rating', 'Netflix Rating', 'Description']:
                    try:
                        val=p[i].text.split(': ')[1]
                    except:
                        print(attr, self.url)
                elif attr=='Description':
                    val=p[i+1].text

                elif attr=='IMDB':
                    attr='imdb_url'
                    val=p[i].find('a')['href']

                elif attr in ['Netflix Rating', 'IMDB Rating']:
                    try:
                        val=p[i].find_all('img')[-1]['title'].split(' ')[2]
                    except:
                        print(attr, self.url)

                attrs[attr.lower()]=val
                
        self.attrs=attrs

In [242]:
entries=[]
for idx, url in enumerate(netflix.object_urls):
    entries.append(Article(url).build())
    if idx%50==0:
        print(idx,len(netflix.object_urls))

0 728
http://usa.newonnetflix.info/info/80048418
http://usa.newonnetflix.info/info/1179373
http://usa.newonnetflix.info/info/70299793
http://usa.newonnetflix.info/info/70281022
http://usa.newonnetflix.info/info/70281319
http://usa.newonnetflix.info/info/217303
http://usa.newonnetflix.info/info/219378
50 728
http://usa.newonnetflix.info/info/70245102
http://usa.newonnetflix.info/info/60029167
http://usa.newonnetflix.info/info/292118
http://usa.newonnetflix.info/info/70174823
http://usa.newonnetflix.info/info/70294792
http://usa.newonnetflix.info/info/60023175
http://usa.newonnetflix.info/info/15856715
http://usa.newonnetflix.info/info/80028277
http://usa.newonnetflix.info/info/80065086
http://usa.newonnetflix.info/info/70242078
100 728
http://usa.newonnetflix.info/info/80091925
http://usa.newonnetflix.info/info/70169911
http://usa.newonnetflix.info/info/70152640
http://usa.newonnetflix.info/info/80049872
http://usa.newonnetflix.info/info/80028645
http://usa.newonnetflix.info/info/701770

KeyboardInterrupt: 

In [244]:
import pandas as pd

In [258]:
media=pd.DataFrame.from_records(entries)
media=media[['cast',
       'certificate', 'date added', 'description', 'director', 'duration',
       'episode length', 'genre',  'imdb rating', 'imdb_url',
       'img_src', 'netflix rating','year']]

In [259]:
media

,cast,certificate,date added,description,director,duration,episode length,genre,imdb rating,imdb_url,img_src,netflix rating,year
0,"Jesse Bradford, Tammin Sursok, Chris Marquette...",R,Unknown Date,An engaged couple gets some advice on staying ...,Leslie Greif,1hr 34m,NaN,"Independent Comedies, Late Night Comedies, Rom...",3.7/10,http://www.imdb.com/title/tt1993391/,https://art-s.nflximg.net/1431c/b848e0ad24ce00...,2.7/5,2013
1,"Amy Smart, Mark-Paul Gosselaar, Jayne Eastwood...",TV-PG,Unknown Date,Set up on a blind date with handsome Miles on ...,James Hayman,1hr 26m,NaN,"Romantic Comedies, Comedies, Romantic Films",6.3/10,http://www.imdb.com/title/tt1846442/,https://art-s.nflximg.net/1d0c0/af6c7181c603a2...,3.9/5,2011
2,"Jennifer Garner, Mark Ruffalo, Judy Greer, And...",PG-13,Unknown Date,When a geeky teen's birthday party goes awry a...,Gary Winick,1hr 37m,NaN,"Romantic Comedies, Comedies, Romantic Films",6.1/10,http://www.imdb.com/title/tt0337563/,https://art-s.nflximg.net/5a872/0c5ed075f2873c...,3.6/5,2004
3,"Lindsey Shaw, Chandler Massey, Keith Coulouris...",PG,Unknown Date,A teen tennis phenom whose life is wrapped up ...,Adam Lipsius,1hr 28m,NaN,"Sports Films, Sports Comedies, Family Features...",5.8/10,http://www.imdb.com/title/tt1663673/,https://art-s.nflximg.net/bda18/5462674525c1eb...,3.8/5,2012
4,"Julie Delpy, Chris Rock, Albert Delpy, Alexia ...",R,Unknown Date,Marion and Mingus go from perfect domesticity ...,Julie Delpy,1hr 35m,NaN,"International Comedies, Independent Comedies, ...",6/10,http://www.imdb.com/title/tt1602472/,https://art-s.nflximg.net/d56db/12a2a8a139ccd2...,3.2/5,2012
5,"Phyllida Law, Don McKellar, Gil Bellows, Chris...",NR,13th April 2016,When he befriends a shady writer during a trip...,"Gil Bellows, Tony Pantages",1hr 22m,NaN,"Dark Comedies, Dramas, Comedies",5.8/10,http://www.imdb.com/title/tt1842371/,https://art-s.nflximg.net/ffc8e/b8d87939602ab0...,3.1/5,2013
6,"Tina Fey, Alec Baldwin, Tracy Morgan, Jane Kra...",TV-14,Unknown Date,"This smartly crafted sitcom follows Liz Lemon,...",NaN,7 Seasons,22min,"Sitcoms, TV Programmes, TV Comedies",8.2/10,http://www.imdb.com/title/tt0496424/,https://art-s.nflximg.net/8e559/6123a8d7009832...,3.8/5,2012
7,"John Lithgow, Kristen Johnston, French Stewart...",TV-PG,4th April 2015,A team of aliens is sent to Earth to probe the...,NaN,6 Seasons,22min,"Sitcoms, TV Sci-Fi & Fantasy, TV Programmes, A...",7.8/10,http://www.imdb.com/title/tt0115082/,https://art-s.nflximg.net/fe7b2/46db18b2c633ed...,4/5,2000
8,"Sergio Freire, Rodrigo Salinas, Ariel Levy, Lo...",NR,20th January 2015,In order to get some respect from their superi...,José Miguel Zúñiga,1hr 30m,NaN,"International Comedies, Latin American Films, ...",3.3/10,http://www.imdb.com/title/tt3465570/,https://art-s.nflximg.net/58ed7/b607c0f5b991c2...,3/5,2014
9,"Ariel Levy, Loreto Aravena, Rodrigo Salinas, S...",TV-14,22nd February 2016,"Bumbling cops Freire and Salinas, now working ...",Ernesto Díaz Espinoza,1hr 33m,NaN,"Spoofs & Satires, Comedies, International Come...",http://www.imdb.com/title/tt4418280/,http://www.imdb.com/title/tt4418280/,https://art-s.nflximg.net/8b9c9/384faf97595e45...,2.8/5,2015


In [ ]:
netflix.object_urls

In [ ]:
#add mapreduce